In [1]:
import pandas as pd
import numpy as np
import json      # library for working with JSON-formatted text strings
import requests  # library for accessing content from web URLs
import pprint    # library for cleanly printing Python data structures
pp = pprint.PrettyPrinter()

import seaborn as sns

In [35]:
response = requests.get("https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2013/race/sex?page=76")
results = response.text
data = json.loads(results)
# data
# pp.pprint(data)
# count = data['count']
# count
ap_2011 = pd.DataFrame(data['results'])
ap_2011


# Replace all missing values in extra columns with 0 
ap_2011.students_AP_exam_all = \
ap_2011.students_AP_exam_all.mask(ap_2011.students_AP_exam_all.lt(0), 0)

ap_2011.students_AP_pass_all = \
ap_2011.students_AP_pass_all.mask(ap_2011.students_AP_pass_all.lt(0), 0)

# Combine extra 2011 columns 
ap_2011['students_AP_exam_oneormore'] = ap_2011['students_AP_exam_oneormore'] + ap_2011['students_AP_exam_all']
ap_2011['students_AP_pass_oneormore'] = ap_2011['students_AP_pass_oneormore'] + ap_2011['students_AP_pass_all']


# Drop unnecessary columns
ap_2011 = ap_2011.drop(labels=['disability', 'lep', 'students_AP_exam_all', 'students_AP_pass_all'], axis=1)


# Label missing values with NaN 
ap_2011.students_AP_exam_none = \
ap_2011.students_AP_exam_none.mask(ap_2011.students_AP_exam_none.lt(0), np.nan)

ap_2011.students_AP_pass_none = \
ap_2011.students_AP_pass_none.mask(ap_2011.students_AP_pass_none.lt(0), np.nan)

ap_2011.students_AP_exam_oneormore = \
ap_2011.students_AP_exam_oneormore.mask(ap_2011.students_AP_exam_oneormore.lt(0), np.nan)

ap_2011.students_AP_pass_oneormore = \
ap_2011.students_AP_pass_oneormore.mask(ap_2011.students_AP_pass_oneormore.lt(0), np.nan)



ap_2011


,crdc_id,ncessch,year,fips,leaid,race,sex,students_AP_exam_none,students_AP_pass_none,students_AP_exam_oneormore,students_AP_pass_oneormore
0,172325002560,172325002560,2011,17,1723250,1,1,0.0,7.0,16.0,NaN
1,172325002560,172325002560,2011,17,1723250,2,1,0.0,NaN,0.0,NaN
2,172325002560,172325002560,2011,17,1723250,3,1,0.0,NaN,0.0,NaN
3,172325002560,172325002560,2011,17,1723250,4,1,0.0,NaN,0.0,NaN
4,172325002560,172325002560,2011,17,1723250,5,1,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2995,172892003035,172892003035,2011,17,1728920,4,1,NaN,NaN,NaN,NaN
2996,172892003035,172892003035,2011,17,1728920,5,1,NaN,NaN,NaN,NaN
2997,172892003035,172892003035,2011,17,1728920,6,1,NaN,NaN,NaN,NaN
2998,172892003035,172892003035,2011,17,1728920,7,1,NaN,NaN,NaN,NaN


In [3]:
count = 2295240 #3000*10
print(count)
if (count % 3000 == 0): 
    max_page = int(count / 3000) 
else: 
    max_page = int(count / 3000) + 1 
max_page 


# 766/30

2295240


766

In [4]:
for i in range(514, 767): 
    print(i)

514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763


In [5]:
ap11_all = pd.DataFrame() 

for i in range(514, 767): 
    api = "https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2011/race/sex?page=" + str(i)
    if (i % 10 == 0):
        print(api)
    response = requests.get(api)
    results = response.text 
    data = json.loads(results)
    ap_2011 = pd.DataFrame(data['results'])
    
    # Replace all missing values in extra columns with 0 
    ap_2011.students_AP_exam_all = \
    ap_2011.students_AP_exam_all.mask(ap_2011.students_AP_exam_all.lt(0), 0)

    ap_2011.students_AP_pass_all = \
    ap_2011.students_AP_pass_all.mask(ap_2011.students_AP_pass_all.lt(0), 0)

    # Combine extra 2011 columns 
    ap_2011['students_AP_exam_oneormore'] = ap_2011['students_AP_exam_oneormore'] + ap_2011['students_AP_exam_all']
    ap_2011['students_AP_pass_oneormore'] = ap_2011['students_AP_pass_oneormore'] + ap_2011['students_AP_pass_all']


    # Drop unnecessary columns
    ap_2011 = ap_2011.drop(labels=['disability', 'lep', 'students_AP_exam_all', 'students_AP_pass_all'], axis=1)


    # Label missing values with NaN 
    ap_2011.students_AP_exam_none = \
    ap_2011.students_AP_exam_none.mask(ap_2011.students_AP_exam_none.lt(0), np.nan)

    ap_2011.students_AP_pass_none = \
    ap_2011.students_AP_pass_none.mask(ap_2011.students_AP_pass_none.lt(0), np.nan)

    ap_2011.students_AP_exam_oneormore = \
    ap_2011.students_AP_exam_oneormore.mask(ap_2011.students_AP_exam_oneormore.lt(0), np.nan)

    ap_2011.students_AP_pass_oneormore = \
    ap_2011.students_AP_pass_oneormore.mask(ap_2011.students_AP_pass_oneormore.lt(0), np.nan)
    
    
    # Put all of our page data frames on top of each other 
    ap11_all = pd.concat([ap11_all, ap_2011]).reset_index(drop=True)
    
    
ap11_all

https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2011/race/sex?page=520
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2011/race/sex?page=530
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2011/race/sex?page=540
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2011/race/sex?page=550
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2011/race/sex?page=560
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2011/race/sex?page=570
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2011/race/sex?page=580
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2011/race/sex?page=590
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2011/race/sex?page=600
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2011/race/sex?page=610
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2011/race/sex?page=620
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2011/race/sex?p

,crdc_id,ncessch,year,fips,leaid,race,sex,students_AP_exam_none,students_AP_pass_none,students_AP_exam_oneormore,students_AP_pass_oneormore
0,010252001800,010252001800,2011,1,0102520,1,99,NaN,NaN,NaN,NaN
1,010252001800,010252001800,2011,1,0102520,2,99,NaN,NaN,NaN,NaN
2,010252001800,010252001800,2011,1,0102520,3,99,NaN,NaN,NaN,NaN
3,010252001800,010252001800,2011,1,0102520,4,99,NaN,NaN,NaN,NaN
4,010252001800,010252001800,2011,1,0102520,5,99,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
756235,568025200350,568025200350,2011,56,5680252,4,99,NaN,None,NaN,NaN
756236,568025200350,568025200350,2011,56,5680252,5,99,NaN,None,NaN,NaN
756237,568025200350,568025200350,2011,56,5680252,6,99,NaN,None,NaN,NaN
756238,568025200350,568025200350,2011,56,5680252,7,99,NaN,None,NaN,NaN


In [6]:
ap11_all.nunique()

crdc_id                       94530
ncessch                       92963
year                              1
fips                             51
leaid                         15979
race                              8
sex                               1
students_AP_exam_none           440
students_AP_pass_none           424
students_AP_exam_oneormore      906
students_AP_pass_oneormore      688
dtype: int64

In [7]:
ap11_all
# 256 -516 -
# ap11_all.to_csv('ap11_all_3.csv') 

In [5]:
response = requests.get("https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2017/race/sex?page=1")
results = response.text
data = json.loads(results)
data
# pp.pprint(data)
# count = data['count']
# count
# ap_page = pd.DataFrame(data['results'])
# ap_page

# # Drop unnecessary columns
# ap_page = ap_page.drop(labels=['disability', 'lep', 'students_AP_exam_all', 'students_AP_pass_all'], axis=1)


# # Label missing values with NaN 
# ap_page.students_AP_exam_none = \
# ap_page.students_AP_exam_none.mask(ap_page.students_AP_exam_none.lt(0), np.nan)

# ap_page.students_AP_pass_none = \
# ap_page.students_AP_pass_none.mask(ap_page.students_AP_pass_none.lt(0), np.nan)

# ap_page.students_AP_exam_oneormore = \
# ap_page.students_AP_exam_oneormore.mask(ap_page.students_AP_exam_oneormore.lt(0), np.nan)

# ap_page.students_AP_pass_oneormore = \
# ap_page.students_AP_pass_oneormore.mask(ap_page.students_AP_pass_oneormore.lt(0), np.nan)


# ap_page

{'count': 2343168,
 'next': 'https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2017/race/sex/?page=2',
 'previous': None,
 'results': [{'crdc_id': '010000201705',
   'ncessch': '010000201705',
   'year': 2017,
   'fips': 1,
   'leaid': '0100002',
   'race': 1,
   'sex': 1,
   'disability': 99,
   'lep': 99,
   'students_AP_exam_none': -2,
   'students_AP_pass_none': None,
   'students_AP_exam_oneormore': -2,
   'students_AP_pass_oneormore': None,
   'students_AP_exam_all': None,
   'students_AP_pass_all': None},
  {'crdc_id': '010000201705',
   'ncessch': '010000201705',
   'year': 2017,
   'fips': 1,
   'leaid': '0100002',
   'race': 2,
   'sex': 1,
   'disability': 99,
   'lep': 99,
   'students_AP_exam_none': -2,
   'students_AP_pass_none': None,
   'students_AP_exam_oneormore': -2,
   'students_AP_pass_oneormore': None,
   'students_AP_exam_all': None,
   'students_AP_pass_all': None},
  {'crdc_id': '010000201705',
   'ncessch': '010000201705',
   'year': 2017,
   'fips'

# Gather other years

In [1]:
import pandas as pd
import numpy as np
import json      # library for working with JSON-formatted text strings
import requests  # library for accessing content from web URLs
import pprint    # library for cleanly printing Python data structures
# pp = pprint.PrettyPrinter()

# import seaborn as sns

In [5]:
count = 2343168 #3000*10
print(count)
if (count % 3000 == 0): 
    max_page = int(count / 3000) 
else: 
    max_page = int(count / 3000) + 1 
max_page 


# 766/30

2343168


782

In [9]:
# for i in range(1, 387): 
# for i in range(3, 772): 
    print(i)

387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636


In [2]:
# Read in the other 3 years 
ap17_all = pd.DataFrame() 

for i in range(392, 783): 
    api = "https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2017/race/sex?page=" + str(i)
    if (i  % 30 == 0):
        print(api)
    response = requests.get(api)
    results = response.text 
    data = json.loads(results)
    ap_page = pd.DataFrame(data['results'])
    

    # Drop unnecessary columns
    ap_page = ap_page.drop(labels=['disability', 'lep', 'students_AP_exam_all', 'students_AP_pass_all'], axis=1)


    # Label missing values with NaN 
    ap_page.students_AP_exam_none = \
    ap_page.students_AP_exam_none.mask(ap_page.students_AP_exam_none.lt(0), np.nan)

    ap_page.students_AP_pass_none = \
    ap_page.students_AP_pass_none.mask(ap_page.students_AP_pass_none.lt(0), np.nan)

    ap_page.students_AP_exam_oneormore = \
    ap_page.students_AP_exam_oneormore.mask(ap_page.students_AP_exam_oneormore.lt(0), np.nan)

    ap_page.students_AP_pass_oneormore = \
    ap_page.students_AP_pass_oneormore.mask(ap_page.students_AP_pass_oneormore.lt(0), np.nan)

    
    # Put all of our page data frames on top of each other 
    ap17_all = pd.concat([ap17_all, ap_page]).reset_index(drop=True)
    
    
ap17_all

https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2017/race/sex?page=420
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2017/race/sex?page=450
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2017/race/sex?page=480
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2017/race/sex?page=510
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2017/race/sex?page=540
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2017/race/sex?page=570
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2017/race/sex?page=600
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2017/race/sex?page=630
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2017/race/sex?page=660
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2017/race/sex?page=690
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2017/race/sex?page=720
https://educationdata.urban.org/api/v1/schools/crdc/ap-exams/2017/race/sex?p

,crdc_id,ncessch,year,fips,leaid,race,sex,students_AP_exam_none,students_AP_pass_none,students_AP_exam_oneormore,students_AP_pass_oneormore
0,291365002554,291365002554,2017,29,2913650,1,2,NaN,None,NaN,None
1,291365002554,291365002554,2017,29,2913650,2,2,NaN,None,NaN,None
2,291365002554,291365002554,2017,29,2913650,3,2,NaN,None,NaN,None
3,291365002554,291365002554,2017,29,2913650,4,2,NaN,None,NaN,None
4,291365002554,291365002554,2017,29,2913650,5,2,NaN,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...
1170163,720003002085,720003002085,2017,72,7200030,4,99,0.0,None,0.0,None
1170164,720003002085,720003002085,2017,72,7200030,5,99,0.0,None,0.0,None
1170165,720003002085,720003002085,2017,72,7200030,6,99,0.0,None,0.0,None
1170166,720003002085,720003002085,2017,72,7200030,7,99,0.0,None,0.0,None


In [3]:
# 383 -- 765 
# ap13_all.to_csv('ap13_all_2.csv') 

# 180 -- 383 -- 765 -- 771
# ap15_all.to_csv('ap15_all_1.5.csv')

# 391 -- 782
ap17_all.to_csv('ap17_all_2.csv')


In [4]:
# 2011 stores the last columns differently than the other three years 
# Combining students_AP_exam_all with students_AP_exam_oneormore
ap_2011 = ap_taken.copy() 
ap11_exam_1more = ap_2011.loc[(ap_2011['students_AP_exam_all'] >= 0) & 
                              (ap_2011['students_AP_exam_oneormore'] >= 0)].copy()

ap11_exam_1more['students_AP_exam_oneormore'] = \
ap11_exam_1more['students_AP_exam_oneormore'] + ap11_exam_1more['students_AP_exam_all']
ap11_exam_1more['students_AP_pass_oneormore'] = \
ap11_exam_1more['students_AP_pass_oneormore'] + ap11_exam_1more['students_AP_pass_all']

# Put all of our data frames on top of each other 
# ap_taken = pd.concat([ap11_exam_1more, ap_2013, ap_2015, ap_2017]).reset_index(drop=True)

# Drop unnecessary columns
ap_taken = ap_taken.drop(labels=['disability', 'lep', 'students_AP_exam_all', 'students_AP_pass_all'], axis=1)


# Filter out missing data in exams taken column 
ap_taken = ap_taken.loc[(ap_taken['students_AP_exam_oneormore'] >= 0)]


# Replace all missing values in passing columns with NaN 
ap_taken.students_AP_pass_oneormore.fillna(value=np.nan, inplace=True)
ap_taken.students_AP_pass_none.fillna(value=np.nan, inplace=True)
ap_taken.students_AP_exam_none.fillna(value=np.nan, inplace=True)


ap_taken.students_AP_pass_oneormore = \
ap_taken.students_AP_pass_oneormore.mask(ap_taken.students_AP_pass_oneormore.lt(0), np.nan)
ap_taken.students_AP_pass_none = \
ap_taken.students_AP_pass_none.mask(ap_taken.students_AP_pass_none.lt(0), np.nan)
ap_taken.students_AP_exam_none = \
ap_taken.students_AP_exam_none.mask(ap_taken.students_AP_exam_none.lt(0), np.nan)

# Change the race column values to be more easily interpretable 
race_dict = {1 : 'White', 2 : 'Black', 3: 'Hispanic', 4 : 'Asian', 
             5: 'Indigenous \nAmerican ', 
             6: "Pacific \nIslander", 
             7 : "Multiracial", 8 : "Nonresident alien", 9 : "Unknown", 
             20 : "Other", 99 : "Total"}
ap_taken = ap_taken.replace({"race": race_dict})

ap_taken

,crdc_id,ncessch,year,fips,leaid,race,sex,students_AP_exam_none,students_AP_pass_none,students_AP_exam_oneormore,students_AP_pass_oneormore
40,010000500871,010000500871,2011,1,0100005,White,1,8,NaN,2,NaN
41,010000500871,010000500871,2011,1,0100005,Black,1,0,NaN,0,NaN
42,010000500871,010000500871,2011,1,0100005,Hispanic,1,0,NaN,2,NaN
43,010000500871,010000500871,2011,1,0100005,Asian,1,0,NaN,0,NaN
44,010000500871,010000500871,2011,1,0100005,Indigenous \nAmerican,1,0,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
29899,040885002552,040885002552,2011,4,0408850,Asian,1,0,NaN,2,NaN
29900,040885002552,040885002552,2011,4,0408850,Indigenous \nAmerican,1,0,NaN,0,NaN
29901,040885002552,040885002552,2011,4,0408850,Pacific \nIslander,1,0,NaN,0,NaN
29902,040885002552,040885002552,2011,4,0408850,Multiracial,1,0,NaN,0,NaN


In [7]:
# Filter out any rows that have missing values in the exams passed column 
ap_passed = ap_passed.loc[(ap_passed['students_AP_pass_oneormore'] >= 0)]


# Set missing values in none passed column to NaN 
ap_passed.students_AP_pass_none.fillna(value=np.nan, inplace=True)
ap_passed.students_AP_pass_none = \
ap_passed.students_AP_pass_none.mask(ap_passed.students_AP_pass_none.lt(0), np.nan)


# Change the race column values to be more easily interpretable 
race_dict = {1 : 'White', 2 : 'Black', 3: 'Hispanic', 4 : 'Asian', 
             5: 'Indigenous \nAmerican ', 
             6: "Pacific \nIslander", 
             7 : "Multiracial", 8 : "Nonresident alien", 9 : "Unknown", 
             20 : "Other", 99 : "Total"}
ap_passed = ap_passed.replace({"race": race_dict})

ap_passed

,crdc_id,ncessch,year,fips,leaid,race,sex,students_AP_exam_none,students_AP_pass_none,students_AP_exam_oneormore,students_AP_pass_oneormore
208,010000700251,010000700251,2011,1,0100007,White,1,23,25.0,118,95
215,010000700251,010000700251,2011,1,0100007,Total,1,32,36.0,162,122
240,010000701456,010000701456,2011,1,0100007,White,1,29,28.0,112,89
247,010000701456,010000701456,2011,1,0100007,Total,1,35,32.0,131,97
336,010000800831,010000800831,2011,1,0100008,White,1,5,37.0,199,164
...,...,...,...,...,...,...,...,...,...,...,...
29880,040885002195,040885002195,2011,4,0408850,White,1,29,13.0,61,44
29882,040885002195,040885002195,2011,4,0408850,Hispanic,1,17,NaN,7,8
29887,040885002195,040885002195,2011,4,0408850,Total,1,53,13.0,77,56
29896,040885002552,040885002552,2011,4,0408850,White,1,8,19.0,43,23


In [3]:
### API PRACTICE ### 
response = requests.get("https://educationdata.urban.org/api/v1/schools/ccd/directory/2019/")

results = response.text
# print(type(results)) 
# print(results[:500])

# # parse the string into a Python dictionary (loads = "load string")
data = json.loads(results)

# print(data[:3])
# print(type(data))

# pp.pprint(data)
# data['results'][0].keys()
ccd = pd.DataFrame(data['results'])
ccd.columns
# ccd['highest_grade_offered']
ccd

,year,ncessch,school_id,school_name,leaid,lea_name,state_leaid,seasch,street_mailing,city_mailing,...,high_cedp,middle_cedp,ungrade_cedp,enrollment,state_leg_district_lower,state_leg_district_upper,ncessch_num,congress_district_id,direct_certification,lunch_program
0,2019,010000500870,00870,Albertville Middle School,0100005,Albertville City,AL-101,101-0010,600 E Alabama Ave,Albertville,...,0,1,0,861.0,01026,01009,10000500870,104,286.0,2.0
1,2019,010000500871,00871,Albertville High School,0100005,Albertville City,AL-101,101-0020,402 E McCord Ave,Albertville,...,1,0,0,1554.0,01026,01009,10000500871,104,425.0,2.0
2,2019,010000500879,00879,Evans Elementary School,0100005,Albertville City,AL-101,101-0110,901 W McKinney Ave,Albertville,...,0,0,0,908.0,01026,01009,10000500879,104,327.0,2.0
3,2019,010000500889,00889,Albertville Elementary School,0100005,Albertville City,AL-101,101-0200,145 West End Drive,Albertville,...,0,0,0,934.0,01026,01009,10000500889,104,368.0,2.0
4,2019,010000501616,01616,Albertville Kindergarten and PreK,0100005,Albertville City,AL-101,101-0035,257 Country Club Rd,Albertville,...,0,0,0,607.0,01026,01009,10000501616,104,254.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2019,040095603505,03505,ASU Preparatory Academy - South Phoenix Primary,0400956,ASU Preparatory Academy (92716),AZ-92716,92716-70692,1130 E University Dr. Ste 230,Tempe,...,0,0,0,115.0,04027,04027,40095603505,407,28.0,1.0
2996,2019,040095803534,03534,BASIS Chandler Primary - North Campus,0400958,BASIS Charter Schools Inc. (92997),AZ-92997,92997-93010,1800 E CHANDLER BLVD,CHANDLER,...,0,0,0,630.0,04017,04017,40095803534,405,NaN,0.0
2997,2019,040095903579,03579,Legacy Traditional School - Phoenix,0400959,Legacy Traditional School - Phoenix (873957),AZ-873957,873957-112572,3201 S GILBERT RD,CHANDLER,...,0,1,0,1429.0,04029,04029,40095903579,403,204.0,1.0
2998,2019,040096000049,00049,Michael Anderson,0400960,Avondale Elementary District (4272),AZ-4272,4272-5347,45 S 3RD AVE,AVONDALE,...,0,1,0,841.0,04019,04019,40096000049,403,316.0,1.0


In [4]:
ccd.nunique()
ccd.value_counts('fips')

fips
1    1556
4     936
2     508
dtype: int64

In [5]:
response = requests.get("https://educationdata.urban.org/api/v1/schools/crdc/enrollment/2011/race/sex/")
results = response.text
data = json.loads(results)
enroll = pd.DataFrame(data['results'])
enroll

,crdc_id,ncessch,year,fips,leaid,race,sex,disability,lep,enrollment_crdc,psenrollment_crdc
0,010000201705,010000201705,2011,1,0100002,1,1,99,99,131,-2
1,010000201705,010000201705,2011,1,0100002,1,2,99,99,0,-2
2,010000201705,010000201705,2011,1,0100002,1,99,99,99,131,-2
3,010000201705,010000201705,2011,1,0100002,2,1,99,99,236,-2
4,010000201705,010000201705,2011,1,0100002,2,2,99,99,0,-2
...,...,...,...,...,...,...,...,...,...,...,...
2995,010018500952,010018500952,2011,1,0100185,7,2,99,99,2,0
2996,010018500952,010018500952,2011,1,0100185,7,99,99,99,4,0
2997,010018500952,010018500952,2011,1,0100185,99,1,99,99,513,7
2998,010018500952,010018500952,2011,1,0100185,99,2,99,99,426,9


In [6]:
enroll.loc[(enroll['enrollment_crdc'] >= 0)]

,crdc_id,ncessch,year,fips,leaid,race,sex,disability,lep,enrollment_crdc,psenrollment_crdc
0,010000201705,010000201705,2011,1,0100002,1,1,99,99,131,-2
1,010000201705,010000201705,2011,1,0100002,1,2,99,99,0,-2
2,010000201705,010000201705,2011,1,0100002,1,99,99,99,131,-2
3,010000201705,010000201705,2011,1,0100002,2,1,99,99,236,-2
4,010000201705,010000201705,2011,1,0100002,2,2,99,99,0,-2
...,...,...,...,...,...,...,...,...,...,...,...
2995,010018500952,010018500952,2011,1,0100185,7,2,99,99,2,0
2996,010018500952,010018500952,2011,1,0100185,7,99,99,99,4,0
2997,010018500952,010018500952,2011,1,0100185,99,1,99,99,513,7
2998,010018500952,010018500952,2011,1,0100185,99,2,99,99,426,9


In [7]:
enroll.nunique()

crdc_id              125
ncessch              119
year                   1
fips                   1
leaid                 39
race                   8
sex                    3
disability             1
lep                    1
enrollment_crdc      440
psenrollment_crdc     51
dtype: int64

In [13]:
response = requests.get("https://educationdata.urban.org/api/v1/schools/crdc/math-and-science/2017/race/sex/?page=2")
results = response.text
data = json.loads(results)
mathsci = pd.DataFrame(data['results'])
mathsci

,crdc_id,year,fips,ncessch,leaid,sex,race,disability,lep,enrl_biology,enrl_chemistry,enrl_advanced_math,enrl_calculus,enrl_algebra2,enrl_physics,enrl_geometry
0,010018802428,2017,1,010018802428,0100188,99,3,99,99,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,010018802428,2017,1,010018802428,0100188,2,1,99,99,-2.0,-2.0,-2.0,-1.0,-1.0,-2.0,-2.0
2,010018802428,2017,1,010018802428,0100188,2,3,99,99,-2.0,-2.0,-2.0,-1.0,-1.0,-2.0,-2.0
3,010018802428,2017,1,010018802428,0100188,1,6,99,99,-2.0,-2.0,-2.0,-1.0,-1.0,-2.0,-2.0
4,010018802428,2017,1,010018802428,0100188,1,99,99,99,-2.0,-2.0,-2.0,-1.0,-1.0,-2.0,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,010039000191,2017,1,010039000191,0100390,2,4,99,99,-2.0,-2.0,-2.0,-1.0,-1.0,-2.0,-2.0
2996,010039000191,2017,1,010039000191,0100390,1,1,99,99,-2.0,-2.0,-2.0,-1.0,-1.0,-2.0,-2.0
2997,010039000191,2017,1,010039000191,0100390,2,3,99,99,-2.0,-2.0,-2.0,-1.0,-1.0,-2.0,-2.0
2998,010039000191,2017,1,010039000191,0100390,99,1,99,99,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
response = requests.get("https://educationdata.urban.org/api/v1/schools/edfacts/grad-rates/2011/")
results = response.text
data = json.loads(results)
grad = pd.DataFrame(data['results'])
grad 

,cohort_num,disability,econ_disadvantaged,fips,foster_care,grad_rate_high,grad_rate_low,grad_rate_midpt,homeless,lea_name,leaid,leaid_num,lep,ncessch,ncessch_num,race,school_name,year
0,268.0,99,99,1,99,83.0,83.0,83.0,99,Albertville City,0100005,100005,99,010000500871,10000500871,99,Albertville High Sch,2011
1,NaN,99,99,1,99,NaN,NaN,NaN,99,Albertville City,0100005,100005,99,010000500871,10000500871,5,Albertville High Sch,2011
2,NaN,99,99,1,99,NaN,NaN,NaN,99,Albertville City,0100005,100005,99,010000500871,10000500871,4,Albertville High Sch,2011
3,6.0,99,99,1,99,100.0,50.0,75.0,99,Albertville City,0100005,100005,99,010000500871,10000500871,2,Albertville High Sch,2011
4,18.0,1,99,1,99,79.0,60.0,69.0,99,Albertville City,0100005,100005,99,010000500871,10000500871,99,Albertville High Sch,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,3.0,99,1,1,99,-3.0,-3.0,-3.0,99,Montgomery County,0102430,102430,99,010243001016,10243001016,99,Georgia Washington Jr High Sch,2011
2996,NaN,99,99,1,99,NaN,NaN,NaN,99,Montgomery County,0102430,102430,99,010243001016,10243001016,3,Georgia Washington Jr High Sch,2011
2997,NaN,99,99,1,99,NaN,NaN,NaN,99,Montgomery County,0102430,102430,1,010243001016,10243001016,99,Georgia Washington Jr High Sch,2011
2998,NaN,99,99,1,99,NaN,NaN,NaN,99,Montgomery County,0102430,102430,99,010243001016,10243001016,7,Georgia Washington Jr High Sch,2011


In [21]:
response = requests.get("https://educationdata.urban.org/api/v1/school-district/edfacts/assessments/2014/9")
results = response.text

# import requests

# response.raise_for_status()  # raises exception when not a 2xx response
# if response.status_code != 204:
#     return response.json()

results
data = json.loads(results)
stass = pd.DataFrame(data['results'])
stass

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# bus = pd.read_csv('data/businesses.csv', encoding='ISO-8859-1')
# apscores = pd.read_excel('project/SchoolYear2020-2021-APScores.xlsx')
# apscores